In [ ]:
!pip install --upgrade jupyterlab-pygments jupyterlab-widgets jupyterlab ipywidgets==7.6.5

In [1]:
# Import Statements
import plotly.graph_objects as go
import plotly.express as px
import vtk
import numpy as np
import ipywidgets as widgets
from IPython.display import clear_output

In [2]:
# Read VTI File
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName('./mixture.vti')
reader.Update()

# Get the VTK data
data = reader.GetOutput()
dims = data.GetDimensions()

In [3]:
# Get Total Number of Points
numOfPoints = data.GetNumberOfPoints()
print(numOfPoints)

421875


In [4]:
# Exploring mixture.vti file
point_data = data.GetPointData()
num_arrays = point_data.GetNumberOfArrays()
array_name = point_data.GetArrayName(0)
print(array_name)

ImageFile


In [5]:
# Get data_values
val_arr = data.GetPointData().GetArray('ImageFile')

In [6]:
# Getting x,y,z and data_values from mixture.vti 
x_values = []
y_values = []
z_values = []
data_values = []
for i in range(numOfPoints):
    x,y,z = data.GetPoint(i)
    x_values.append(x)
    y_values.append(y)
    z_values.append(z)
    data_values.append(val_arr.GetValue(i))
    
print(len(x_values),len(y_values),len(z_values),len(data_values))

421875 421875 421875 421875


In [7]:
# Get the range of data_values
data_min = np.min(data_values)
data_max = np.max(data_values)
print(data_min,data_max)

-0.9935540556907654 0.43280163407325745


In [17]:
# Making Isosurface object
isosurface_data = go.Isosurface(
    x = x_values,
    y = y_values,
    z = z_values,
    value = data_values,
    colorscale='plasma',   # Change colorscale here
    isomin = 0,
    isomax = 0,
    showscale = False,
    cmin = data_min,
    cmax = data_max
)


In [18]:
# Histogram Plot
histogram_data = go.Histogram(x=data_values,nbinsx=30)

In [20]:
# Isosurface FigureWidget
isosurface_figure_data = go.FigureWidget(data=isosurface_data)
isosurface_figure_data.update_layout(autosize=False, 
                    width=500,
                    height=500,
                    scene=dict(
                        xaxis = dict(
                            showticklabels = False
                        ),
                        yaxis = dict(
                            showticklabels = False
                        ),
                        zaxis = dict(
                            showticklabels = False
                        )
                     )           
                    )

# Histogram Figure Widget
histogram_figure_data = go.FigureWidget(data = [histogram_data])
histogram_figure_data.update_layout(
    xaxis_title="Vortex scalar values",
    yaxis_title="Frequency",
    showlegend=False,
    autosize=False, 
    width=500, 
    height=500, 
)

# Float Slider
slider = widgets.FloatSlider(
    value = 0.0,
    min = data_min,
    max = data_max,
    step = 0.01,
    description='Isoval : ',
    continuous_update=False,
    layout={'width': '400px'}
)

# Reset Button
button = widgets.Button(description="Reset")

# Function for changing slide
def update_isosurface(change):
    isovalue = slider.value
    isosurface_figure_data.update_traces(isomin=isovalue, isomax=isovalue)
    # Change Histogram Data
    ## Fetch range of isovalue = x then show (x-0.25)<=histo<=x+0.25
    filtered_value = []
    for val in data_values:
        if val>=(isovalue - 0.25) and val<=(isovalue+0.25):
            filtered_value.append(val)
    ## Update the histogram
    histogram_figure_data.update_traces(x = filtered_value)

# Function for reset button
def reset_handler(change):
    slider.value = 0.0
    isosurface_figure_data.update_traces(isomin=slider.value, isomax=slider.value)
    ## Update the histogram
    histogram_figure_data.update_traces(x = data_values)

# Attach the event handler function to the slider's value change event
slider.observe(update_isosurface, 'value')

# Attach the event handler function to the buttons
button.on_click(reset_handler)

# Creating Layout
top_part = widgets.HBox([slider,button])
bottom_part = widgets.HBox([isosurface_figure_data,histogram_figure_data])
main_part = widgets.VBox([top_part,bottom_part])
display(main_part)